In [1]:
    from wtypes import *
    import pytest, anyconfig
    invalid = pytest.raises(ValidationError)

In [2]:
    def test_update_item():
        for cls in [Dict, Munch]:
            d = cls[{'a': int}]()
            with invalid: d['a'] = 'abc'
            assert 'a' not in d
            d['a'] = 10
            assert d['a'] == 10

In [3]:
    def test_update_attr():
        for cls in [Munch, DataClass]:
            d = cls[{'a': int}]()
            with invalid: d.a = 'abc'
            assert 'a' not in d
            d.a = 10
            assert d.a == 10

In [4]:
    def test_load_config():
        class BadProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Integer}

        with invalid:
            BadProject().load('pyproject.toml')
        
        class PyProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Dict}
        PyProject().load('pyproject.toml')

In [5]:
    def test_tuple():
        assert Tuple((1,2))
        assert Tuple([1,2])
        with invalid: Tuple[Integer, String]({})

In [6]:
    def test_dict():
        class Thing(Dict):
            a: Integer

        assert Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')


In [7]:
    def test_bunch():
        class Thing(Bunch):
            a: Integer

        assert Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        t = Thing(a=1)
        assert t.a == 1
        with invalid:
            Thing(a='abc')


In [8]:
    def test_dataclass():
        class Thing(DataClass):
            a: Integer

        assert dataclasses.is_dataclass(Thing)
        assert Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')
